## Computer vision 
### Detect the marked boxes

In [1]:
import cv2 as cv
import numpy as np
import glob
import os
import pickle
import matplotlib.pyplot as plt

In [4]:
base_folder = 'images\\' # change this on your machine
images = glob.glob(os.path.join(base_folder, "image_*.jpg")) 
char_to_index = {'A': 0, 'B': 1, 'C': 2, 'D': 3}

In [5]:
images

['images\\image_21.jpg',
 'images\\image_22.jpg',
 'images\\image_23.jpg',
 'images\\image_24.jpg',
 'images\\image_25.jpg']

In [4]:
# we assume that we have run an algorithm for finding horizontal and vertical lines 
# for the two frames (left - MATEMATICA and right - INFORMATICA or FIZICA) in the image

# show this on one image
image_name = 'image_21'

# load vertical lines
vertical_lines_left = pickle.load(open((base_folder + image_name + '_vertical_lines_left.pkl'), 'rb'))
print('Vertical lines for the left frame:\n', vertical_lines_left)
vertical_lines_right = pickle.load(open((base_folder + image_name + '_vertical_lines_right.pkl'), 'rb'))

# load horizontal lines
horizontal_lines_left = pickle.load(open((base_folder + image_name + '_horizontal_lines_left.pkl'), 'rb'))
print('Horizontal lines for the left frame:\n', horizontal_lines_left)
horizontal_lines_right = pickle.load(open((base_folder + image_name + '_horizontal_lines_right.pkl'), 'rb'))

Vertical lines for the left frame:
 [[(901, 0), (901, 2808)], [(1046, 0), (1046, 2808)], [(1186, 0), (1186, 2808)], [(1331, 0), (1331, 2808)], [(1479, 0), (1479, 2808)]]
Horizontal lines for the left frame:
 [[(0, 941), (2066, 941)], [(0, 1055), (2066, 1055)], [(0, 1171), (2066, 1171)], [(0, 1288), (2066, 1288)], [(0, 1398), (2066, 1398)], [(0, 1511), (2066, 1511)], [(0, 1625), (2066, 1625)], [(0, 1738), (2066, 1738)], [(0, 1851), (2066, 1851)], [(0, 1965), (2066, 1965)], [(0, 2081), (2066, 2081)], [(0, 2195), (2066, 2195)], [(0, 2308), (2066, 2308)], [(0, 2418), (2066, 2418)], [(0, 2531), (2066, 2531)], [(0, 2651), (2066, 2651)]]


In [5]:
# print the vertical lines in the left frame (MATEMATICA)
image = cv.imread(base_folder + image_name + '.jpg')
print(image.shape)

grayscale_image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
orig_h, orig_w = grayscale_image.shape  
grayscale_image = grayscale_image[int(0.4 * orig_h):int(0.88 * orig_h)] 

left_image = grayscale_image[:, :int(orig_w * 0.5)].copy()
left_image_color = np.dstack((left_image, left_image, left_image))

for i in range(len(vertical_lines_left)):
    cv.line(left_image_color,vertical_lines_left[i][0],vertical_lines_left[i][1],(0,0,255),5)

cv.imshow("left_image_color", cv.resize(left_image_color, (0, 0), fx=0.15, fy=0.15))
cv.waitKey(0)
cv.destroyAllWindows()

(5847, 4132, 3)


In [6]:
# print the horizontal lines in the left frame (MATEMATICA)
for i in range(len(horizontal_lines_left)):
     # write your code here

cv.imshow("left_image_color", cv.resize(left_image_color, (0, 0), fx=0.15, fy=0.15))
cv.waitKey(0)
cv.destroyAllWindows()

In [7]:
# print the vertical and horizontal lines in the right frame (INFORMATICA or FIZICA)
right_image = grayscale_image[:, int(orig_w * 0.5):].copy()
# write your code here
    
cv.imshow("right_image_color", cv.resize(right_image_color, (0, 0), fx=0.15, fy=0.15))
cv.waitKey(0)
cv.destroyAllWindows()

In [8]:
# load the ground truth
ground_truth_content = np.loadtxt(os.path.join(base_folder, '%s.txt' % image_name), dtype=str)
# obtain the correct answers for the left frame (MATEMATICA)
ground_truth_left = ground_truth_content[1:16]
# print the correct answers for the left frame (MATEMATICA)
print(ground_truth_left)

[['1' 'B']
 ['2' 'B']
 ['3' 'A']
 ['4' 'B']
 ['5' 'C']
 ['6' 'A']
 ['7' 'B']
 ['8' 'D']
 ['9' 'C']
 ['10' 'C']
 ['11' 'A']
 ['12' 'C']
 ['13' 'B']
 ['14' 'D']
 ['15' 'C']]


In [9]:
# obtain the correct answers for the right frame (INFORMATICA or FIZICA)
ground_truth_right = ground_truth_content[16:-1]
# print the correct answers for the right frame (INFORMATICA or FIZICA)
print(ground_truth_right)

[['16' 'C']
 ['17' 'B']
 ['18' 'A']
 ['19' 'C']
 ['20' 'A']
 ['21' 'C']
 ['22' 'B']
 ['23' 'C']
 ['24' 'A']
 ['25' 'D']
 ['26' 'D']
 ['27' 'D']
 ['28' 'B']
 ['29' 'C']
 ['30' 'A']]


In [10]:
# write a function that plots the patches containing X with green
# and patches containing blanks with red
# use here the ground thruth (train mode)
# return the mean pixel value of each patch containing X or a blank

def find_x_from_gt(grayscale_image, vertical_lines, horizontal_lines, ground_truth):
    # grayscale_image - input image containing the frame
    # vertical_lines - list with the vertical lines
    # horizontal_lines - list with horizontal lines
    # ground_truth - grounth truth content for a frame    
    mean_x = []
    mean_blank = [] 
    
    image = np.dstack((grayscale_image, grayscale_image, grayscale_image))
    x_color = (0, 255, 0)  # plot a patch containing an X with green color
    blank_color = (0, 0, 255)  # plot a patch containing a blank with red color    
            
    # crop each patch and display it
    for i in range(len(horizontal_lines) - 1):
        for j in range(len(vertical_lines) - 1):
            x_min = vertical_lines[j][0][0] + 15
            x_max = vertical_lines[j + 1][1][0] - 5
            y_min = horizontal_lines[i][0][1] + 15
            y_max = horizontal_lines[i + 1][1][1] - 5
            
            # write your code here
            
            cv.rectangle(image, (x_min, y_min), (x_max, y_max), color=color, thickness=5)
            cv.putText(image, str(mean_patch)[:3] ,(x_min + 10, y_min + 50), cv.FONT_HERSHEY_COMPLEX, 1, (0,0,0), 2) 
        
    cv.imshow('image result', cv.resize(image, (0, 0), fx=0.4, fy=0.3))
    cv.waitKey(0)
    cv.destroyAllWindows()  
        
    return mean_x, mean_blank

In [11]:
# train a simple classifier on images 21, 22, 23, 24
# test this classifier on image 25
mean_x_values = []
mean_blank_values = []
for i in [21, 22, 23, 24]: 
    image_name = 'image_' + str(i)
   
    print(image_name)
    # load image
    image = cv.imread(base_folder + image_name + '.jpg')
    grayscale_image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    orig_h, orig_w = grayscale_image.shape  
    grayscale_image = grayscale_image[int(0.4 * orig_h):-int(0.12 * orig_h)] 
    left_image = grayscale_image[:, :int(orig_w * 0.5)]
    right_image = grayscale_image[:, int(orig_w * 0.5):]
    
    # load vertical lines
    vertical_lines_left = pickle.load(open((base_folder + image_name + '_vertical_lines_left.pkl'), 'rb'))
    vertical_lines_right = pickle.load(open((base_folder + image_name + '_vertical_lines_right.pkl'), 'rb'))
    # load horizontal lines 
    horizontal_lines_left = pickle.load(open((base_folder + image_name + '_horizontal_lines_left.pkl'), 'rb'))
    horizontal_lines_right = pickle.load(open((base_folder + image_name + '_horizontal_lines_right.pkl'), 'rb'))
    
    # load ground truth
    ground_truth_content = np.loadtxt(os.path.join(base_folder, '%s.txt' % image_name), dtype=str)
    ground_truth_left = ground_truth_content[1:16]
    ground_truth_right = ground_truth_content[16:-1]
    
    mean_x, mean_blank = find_x_from_gt(left_image, vertical_lines_left, horizontal_lines_left, ground_truth_left)
    mean_x_values.extend(mean_x)
    mean_blank_values.extend(mean_blank)
    
    mean_x, mean_blank = find_x_from_gt(right_image, vertical_lines_right, horizontal_lines_right, ground_truth_right)
    mean_x_values.extend(mean_x)
    mean_blank_values.extend(mean_blank)    

image_21
image_22
image_23
image_24


In [ ]:
# build your classifier here

In [16]:
# write a function that plots the patches containing X with green
# and patches containing blanks with red
# use here your classifier (test mode)

In [17]:
# test your classifier on image 25
i = 25
image_name = 'image_' + str(i)
   
print(image_name)
# load image
image = cv.imread(base_folder + image_name + '.jpg')
grayscale_image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
orig_h, orig_w = grayscale_image.shape  
grayscale_image = grayscale_image[int(0.4 * orig_h):-int(0.12 * orig_h)] 
left_image = grayscale_image[:, :int(orig_w * 0.5)]
right_image = grayscale_image[:, int(orig_w * 0.5):]
    
# load vertical lines
vertical_lines_left = pickle.load(open(os.path.join(base_folder, '%s_vertical_lines_left.pkl' % image_name), 'rb'))
vertical_lines_right = pickle.load(open(os.path.join(base_folder, '%s_vertical_lines_right.pkl' % image_name), 'rb'))
# load horizontal lines 
horizontal_lines_left = pickle.load(open(os.path.join(base_folder, '%s_horizontal_lines_left.pkl' % image_name), 'rb'))
horizontal_lines_right = pickle.load(open(os.path.join(base_folder, '%s_horizontal_lines_right.pkl' % image_name), 'rb'))  
#write your code here

image_25
